##### Installing and importing libraries

In [2]:
import requests
import pandas as pd
import numpy as np
import random

!pip install geopy
from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

!pip install folium==0.5.0
import folium

print('Folium installed')
print('Libraries imported')

Folium installed
Libraries imported


#### Scraping Wikipedia page and creating dataframe

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
results

<Response [200]>

In [4]:
tables = pd.read_html(url)

In [5]:
dfprev = tables[0]
dfprev.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Transforming the data

In [6]:
dfprev.rename(columns = {'Neighbourhood': 'Neighborhood', 'Postal Code':'PostalCode'}, inplace = True)
dfprev.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
df = dfprev[dfprev['Borough']!='Not assigned']
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
df.groupby(['PostalCode'])
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
df.shape

(103, 3)

In [10]:
!pip install geocoder
import geocoder # import geocoder

In [11]:
file = 'Geospatial_Coordinates.csv'
coord = pd.read_csv(file)
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_merged = pd.merge(df,coord,left_on='PostalCode',right_on='Postal Code')

In [17]:
df_merged.drop('Postal Code',1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### Mapping Neighborhoods in Canada

In [31]:
latitude = 43.651070
longitude = -79.347015
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'],df_merged['Longitude'],df_merged['Borough'],df_merged['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(map_toronto)
map_toronto

#### Clustering by distance using plugins.MarkerCluster

In [41]:
from folium import plugins
latitude = 43.651070
longitude = -79.347015
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)
incidents = plugins.MarkerCluster().add_to(map_toronto)
for lat, lng, borough, neighborhood, toronto in zip(df_merged['Latitude'],df_merged['Longitude'],df_merged['Borough'],df_merged['Neighborhood'],df_merged['Toronto']):
    label='{},{},{}'.format(neighborhood,borough,toronto)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(incidents)
map_toronto